# This notebook is to test a single batch run in ADAM

In [9]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import AdamProcessingService
from adam import BatchPropagationResults
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [10]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager().get_config('adam_dev')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])
aps = AdamProcessingService(auth_rest)

## Example Inputs

In [11]:
# Keplerian elements state vector
keplerian_elements = {
    'semi_major_axis_km': 448793612,
    'eccentricity': 0.1,
    'inclination_deg': 90,
    'ra_of_asc_node_deg': 91,
    'arg_of_pericenter_deg': 92,
    'true_anomaly_deg': 93,
    'gm': 132712440041.9394
}

# Uncertainties on elements
keplerian_sigma = {
    'semi_major_axis': 100,
    'eccentricity': 0.001,
    'inclination': 1,
    'ra_of_asc_node': 2,
    'arg_of_pericenter': 3,
    'true_anomaly': 4,
}

state_vec = [-150874809.2,
             -187234595.3,
             -73785026.7,
             14.64403935,
             -11.75744819,
             -5.583528281]

cartesian_sigma = {
    "x": 10,
    "y": 10,
    "z": 10,
    "x_dot": 1,
    "y_dot": 1,
    "z_dot": 1    
}


### Set Parameters

Commented parameters are optional. Uncomment to use.

In [12]:
propagation_params = PropagationParams({
    'start_time': '2001-01-02T01:13:46.62Z',  # propagation start time in ISO format
    'end_time': '2010-12-31T01:13:46.62Z',  # propagation end time in ISO format

    #'keplerianSigma': keplerian_sigma,
    'description': 'Jupyter Example',
    'cartesianSigma' : cartesian_sigma,
    'monteCarloDraws': 10,
    'propagationType': 'MONTE_CARLO',
    'stopOnImpact': True,
    'step_size': 86400,
    'stopOnCloseApproach': False,
    "stopOnImpactDistanceMeters": 500000,
    "closeApproachRadiusFromTargetMeters": 7000000000

})

opm_params = OpmParams({
    'epoch': '2001-01-02T01:13:46Z',
    # 'keplerian_elements': keplerian_elements,
    'state_vector': state_vec,
})


### Submit and Run Propagation

In [16]:
batch_run = aps.execute_batch_propagation(config['workspace'], propagation_params, opm_params)
# batch_run = BatchPropagationResults.fromRESTwithRawIds(auth_rest, config['workspace'], 'efaa4ecc-c6d0-4d57-a769-cd400a3a0997')
print(batch_run)

{'job_uuid': 'efaa4ecc-c6d0-4d57-a769-cd400a3a0997', 'results': None}


### Get Status

In [17]:
print(batch_run.check_status())
batch_run.wait_for_complete(print_waiting = True)
print()
print(batch_run.check_status())

CREATED
CREATED
...........RUNNING
.....................
COMPLETED


### Get Number of Runs

In [39]:
# Get ephemeris of specified part
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

# Runs: 11


In [40]:
### Get Summary Statistics

In [42]:
stats = batch_run.get_summary()
print(stats)

{'misses': 10, 'close_approach': 3, 'impacts': 1, 'total': 14, 'pc': 0.07142857142857142}


### Get Ephemeris for specific run

In [43]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

https://storage.googleapis.com/adam-monte-carlo-v1-beam/output/job/e295b498-3e1b-4df4-95b2-fe0ee5d22bc2/stk-ephemerides/MISS/run-8-00000-of-00001.e
stk.v.11.0
BEGIN Ephemeris
ScenarioEpoch 02 Jan 2001 01:13:46.620000
CentralBody SUN
CoordinateSystem ICRF
InterpolationMethod HERMITE
InterpolationOrder 5
NumberOfEphemerisPoints 3652

EphemerisTimePosVel
0.000000000000e+00 -1.508747893621e+11 -1.872345814136e+11 -7.378503263536e+10 1.386212938832e+04 -1.157060183698e+04 -4.559727115700e+03
8.640000000000e+04 -1.496724199376e+11 -1.882284462389e+11 -7.417669355364e+10 1.397032208390e+04 -1.143544636783e+04 -4.506465525638e+03
1.728000000000e+05 -1.484607477360e+11 -1.892106123932e+11 -7.456374430100e+10 1.407747204910e+04 -1.129979949533e+04 -4.453010287077e+03
2.592000000000e+05 -1.472398626990e+11 -1.901810374795e+11 -7.494616817003e+10 1.418358272217e+04 -1.116366261275e+04 -4.399361949764e+03
3.456000000000e+05 -1.460098544782e+11 -1.911396792183e+11 -7.532394849961e+10 1.428865739746e

### Get ending state vector

In [44]:
# Get the end state vector (uncomment to use)
end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
print(f'First close end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
print(len(end_state_vectors))
print(f'First impact end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.MISS)
print("Miss State Vectors")
print(len(end_state_vectors))
print(f'First miss end state: {end_state_vectors[0]}')


Close State Vectors
First close end state: [138335374118.59653, 51196673879.72533, 22195699580.749737]
Impact State Vectors
1
First impact end state: [775389611553.0438, 1652126811982.3506, -1143538336690.8086]
Miss State Vectors
10
First miss end state: [-187317751.3216082, -149033989.55021822, -60684167.585229866]
